In [78]:
import nltk 
import string 
import re 
import pandas as pd
import boto3
import pickle
from sklearn.neighbors import KDTree
import io
import numpy as np

import sagemaker
from sagemaker import get_execution_role
import json

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = 'inroads-blazingtext' # Replace with your own bucket name if needed
print(bucket)
prefix = 'blazingtext/supervised'

arn:aws:iam::068949824886:role/service-role/AmazonSageMaker-ExecutionRole-20200617T154104
inroads-blazingtext


In [ ]:
#download labeled data

In [65]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/labeled_data.csv')
labeled_data = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

labeled_data.shape

(10146, 2)

In [ ]:
#format for blazing text

In [74]:
train_op_data=[]
for i in range(0,len(labeled_data)):
    label = "_label_" + str(labeled_data['cluster'][i])
    train_op_data.append(label)
len(train_op_data)
train_op_data

['_label_2',
 '_label_2',
 '_label_3',
 '_label_1',
 '_label_4',
 '_label_3',
 '_label_3',
 '_label_2',
 '_label_1',
 '_label_4',
 '_label_1',
 '_label_3',
 '_label_2',
 '_label_5',
 '_label_2',
 '_label_3',
 '_label_2',
 '_label_3',
 '_label_1',
 '_label_5',
 '_label_8',
 '_label_1',
 '_label_5',
 '_label_2',
 '_label_8',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_8',
 '_label_0',
 '_label_0',
 '_label_8',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_1',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_1',
 '_label_3',
 '_label_1',
 '_label_2',
 '_label_5',
 '_label_2',
 '_label_1',
 '_label_8',
 '_label_3',
 '_label_2',
 '_label_1',

In [76]:
labeled_data['op_data']=train_op_data
txt=labeled_data['op_data']+" "+labeled_data['token']
txt[0]

'_label_2 abstract'

In [ ]:
## format 

In [80]:
region_name = boto3.Session().region_name

container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:latest (us-east-1)


In [82]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [86]:
%%time

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

#sess.upload_data(path='dbpedia.train', bucket=bucket, key_prefix=train_channel)
#sess.upload_data(path='dbpedia.validation', bucket=bucket, key_prefix=validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 12.2 µs


In [83]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [85]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=1)

In [87]:
bt_model.fit(inputs=txt, logs=True)

ValueError: Cannot format input 0            _label_2 abstract
1             _label_2 absence
2           _label_3 abundance
3             _label_1 reality
4           _label_4 beautiful
                 ...          
10141           _label_4 saudi
10142          _label_6 arabia
10143        _label_1 adultery
10144        _label_8 indecent
10145    _label_6 islamisation
Length: 10146, dtype: object. Expecting one of str, dict, s3_input or FileSystemInput

pandas.core.series.Series